In [1]:
import os
import pickle

# Set your directory path (adjust based on your dataset folder name)
pkl_dir = "/kaggle/input/household-v2"

# List all .pkl files
pkl_files = [f for f in os.listdir(pkl_dir) if f.endswith('.pkl')]

# Load all pkl files into a list or dict
data_list = []

for file in pkl_files:
    file_path = os.path.join(pkl_dir, file)
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        data_list.append(data)

# Optional: Load into a dictionary using filename as key
# data_dict = {file: pickle.load(open(os.path.join(pkl_dir, file), 'rb')) for file in pkl_files}


In [3]:
import pandas as pd

# Combine into one big DataFrame
combined_df = pd.concat(data_list, ignore_index=True)


In [4]:
print(combined_df.shape)
print(combined_df.dtypes)
combined_df.head()


(20029110, 53)
store_id          object
item_id           object
wm_yr_wk          object
d                 object
id                object
dept_id           object
cat_id            object
state_id          object
sales             object
date              object
weekday           object
wday              object
month              int32
year               int32
event_name_1      object
event_type_1      object
event_name_2      object
event_type_2      object
snap_CA           object
snap_TX           object
snap_WI           object
sell_price        object
day_of_week       object
is_weekend         int32
event_type        object
snap_flag         object
sales_mean_3     float64
sales_min_3       object
sales_max_3       object
sales_std_3      float64
sales_mean_7     float64
sales_min_7       object
sales_max_7       object
sales_std_7      float64
sales_mean_28    float64
sales_min_28      object
sales_max_28      object
sales_std_28     float64
price_mean_3     float64
price_min_

store_id          item_id wm_yr_wk      d                               id  \
0     CA_1  HOUSEHOLD_2_505    11213  d_451  HOUSEHOLD_2_505_CA_1_validation   
1     CA_1  HOUSEHOLD_2_505    11213  d_452  HOUSEHOLD_2_505_CA_1_validation   
2     CA_1  HOUSEHOLD_2_505    11213  d_453  HOUSEHOLD_2_505_CA_1_validation   
3     CA_1  HOUSEHOLD_2_505    11213  d_454  HOUSEHOLD_2_505_CA_1_validation   
4     CA_1  HOUSEHOLD_2_505    11213  d_455  HOUSEHOLD_2_505_CA_1_validation   

       dept_id     cat_id state_id sales        date  ... price_min_7  \
0  HOUSEHOLD_2  HOUSEHOLD       CA     0  2012-04-23  ...        4.97   
1  HOUSEHOLD_2  HOUSEHOLD       CA     0  2012-04-24  ...        4.97   
2  HOUSEHOLD_2  HOUSEHOLD       CA     0  2012-04-25  ...        4.97   
3  HOUSEHOLD_2  HOUSEHOLD       CA     2  2012-04-26  ...        4.97   
4  HOUSEHOLD_2  HOUSEHOLD       CA     1  2012-04-27  ...        4.97   

  price_max_7  price_std_7  price_mean_28 price_min_28 price_max_28  \
0        4.97          0.0           4.97         4.97         4.97   
1        4.97          0.0           4.97         4.97         4.97   
2        4.97          0.0           4.97         4.97         4.97   
3        4.97          0.0           4.97         4.97         4.97   
4        4.97          0.0           4.97         4.97         4.97   

  price_std_28 daily_sales daily_price    row_id  
0          0.0           0        4.97  19800001  
1          0.0           0        4.97  19800002  
2          0.0           0        4.97  19800003  
3          0.0           2        4.97  19800004  
4          0.0           1        4.97  19800005  

[5 rows x 53 columns]

In [5]:
from datetime import datetime

# 6. Define date windows
train_end = '2016-04-24'
valid_start = '2016-04-10'     # <-- must be ≤ 2016-04-24
valid_end = '2016-04-24'
sequence_length = 28

# First, convert 'date' to datetime if not already
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Define the earliest date needed to cover the sequence length
min_date = pd.to_datetime(valid_start) - pd.Timedelta(days=sequence_length)

# Filter the dataset to include only the date range needed
filtered_df = combined_df[
    (combined_df['date'] >= min_date) &
    (combined_df['date'] <= pd.to_datetime(train_end))
].copy()


In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder


# ✅ 2. Make sure date is datetime (you already did this earlier, but safe to ensure)
filtered_df['date'] = pd.to_datetime(filtered_df['date'])

# ✅ 3. Define feature columns
categorical_cols = ['event_type', 'is_weekend', 'snap_flag', 'day_of_week', 'month', 'year', 'state_id', 'store_id']
numeric_cols = [
    'sales_mean_7', 'sales_mean_28', 'sales_max_7', 'sales_max_28',
    'sales_min_7', 'sales_min_28', 'sales_std_7', 'sales_std_28',
    'price_mean_7', 'price_mean_28', 'price_max_7', 'price_max_28',
    'price_min_7', 'price_min_28', 'price_std_7', 'price_std_28',
    'sales', 'sell_price'
]

# ✅ 4. Label encode categorical columns
for col in categorical_cols:
    filtered_df[col] = LabelEncoder().fit_transform(filtered_df[col].astype(str))

# ✅ 5. Convert numeric columns safely
for col in numeric_cols:
    filtered_df[col] = pd.to_numeric(filtered_df[col], errors='coerce')

# ✅ 6. Normalize features
feature_cols = categorical_cols + numeric_cols
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(filtered_df[feature_cols])

# ✅ 7. Final scaled DataFrame
scaled_df = pd.DataFrame(scaled_values, columns=feature_cols)
scaled_df['sales'] = filtered_df['sales'].values  # optional, but often useful
scaled_df['date'] = filtered_df['date'].values
scaled_df['d'] = filtered_df['d'].values


In [7]:
# 7. Slice train and validation sets
# ✅ Include 28-day buffer before valid_start
train_df = scaled_df[
    (scaled_df['date'] >= '2015-04-25') & (scaled_df['date'] <= train_end)
].reset_index(drop=True)

valid_df = scaled_df[
    (scaled_df['date'] >= '2016-03-28') & (scaled_df['date'] <= '2016-05-22')
].reset_index(drop=True)

print("Train shape:", train_df.shape)
print("Valid shape:", valid_df.shape)

train_df = train_df.dropna().reset_index(drop=True)
print("✅ Train shape (after dropna):", train_df.shape)

# 8. Memory-efficient training sequence generator
def sequence_generator_multistep(df, input_len=28, output_len=28, batch_size=32, target_col='sales'):
    feature_cols = [col for col in df.columns if col not in ['date', 'd']]
    target_idx = feature_cols.index(target_col)
    data_array = df[feature_cols].values.astype('float32')
    i = input_len
    while True:
        X_batch, y_batch = [], []
        for _ in range(batch_size):
            if i + output_len >= len(data_array):
                i = input_len  # restart
            X_seq = data_array[i-input_len:i]
            y_seq = data_array[i:i+output_len, target_idx]  # grab 28 days future
            X_batch.append(X_seq)
            y_batch.append(y_seq)
            i += 1
        yield np.array(X_batch), np.log1p(np.array(y_batch))  # log transform y

# 9. Fast validation sequence creator
def create_multistep_sequences(df, input_len=28, output_len=28, target_col='sales', min_target_date=None):
    feature_cols = [col for col in df.columns if col not in ['date', 'd']]
    target_idx = feature_cols.index(target_col)

    data_array = df[feature_cols].values.astype('float32')
    date_array = df['date'].values

    X, y = [], []
    for i in range(input_len, len(data_array) - output_len):
        if min_target_date is not None and date_array[i] < min_target_date:
            continue
        X_seq = data_array[i-input_len:i]
        y_seq = data_array[i:i+output_len, target_idx]
        X.append(X_seq)
        y.append(y_seq)
    return np.array(X, dtype='float32'), np.log1p(np.array(y, dtype='float32'))




# 10. Create validation set
X_valid, y_valid = create_multistep_sequences(
    valid_df,
    input_len=28,
    output_len=28,
    min_target_date=pd.to_datetime(valid_start)
)

print(X_valid.shape)  # (num_samples, 28, num_features)
print(y_valid.shape)  # (num_samples, 28)

# ✅ 建立一小段 training set for memory-safe training generator
# 你不需要事先創建 X_train / y_train，全靠 generator 即可
# 但 y_valid 要先轉換：
y_valid = np.log1p(y_valid)  # ✅ 避免 loss 爆炸


Train shape: (450210, 28)
Valid shape: (293160, 28)
✅ Train shape (after dropna): (450210, 28)
(157020, 28, 26)
(157020, 28)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import numpy as np

# ✅ 1. Generator 改為多步輸出
def sequence_generator_multistep(df, input_len=28, output_len=28, batch_size=32, target_col='sales'):
    feature_cols = [col for col in df.columns if col not in ['date', 'd']]
    target_idx = feature_cols.index(target_col)
    data_array = df[feature_cols].values.astype('float32')
    i = input_len
    while True:
        X_batch, y_batch = [], []
        for _ in range(batch_size):
            if i + output_len >= len(data_array):
                i = input_len
            X_seq = data_array[i-input_len:i]
            y_seq = data_array[i:i+output_len, target_idx]
            if np.isnan(X_seq).any() or np.isnan(y_seq).any():
                print(f"🚨 NaN at step {_}")
            X_batch.append(X_seq)
            y_batch.append(y_seq)
            i += 1
        yield np.array(X_batch), np.log1p(np.array(y_batch))  # log1p 避免 loss 爆炸

# ✅ 2. 模型輸出 28 維（你預測 28 天）
n_features = 26  # 記得根據你的 input features 調整
model = Sequential([
    GRU(64, input_shape=(28, n_features), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(28)  # 改成 28-day forecast
])


model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error')

# ✅ 3. Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint_cb = ModelCheckpoint("best_gru_model.keras", monitor="val_loss", save_best_only=True)

# ✅ 4. 訓練設定
batch_size = 16
train_gen = sequence_generator_multistep(train_df, 28, 28, batch_size=batch_size)
num_samples = len(train_df) - 28 - 28
steps_per_epoch = num_samples // batch_size


# ✅ 5. 不要重複 log1p，直接用 create_sequences_fast return 的結果
history = model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stop, checkpoint_cb],
    verbose=1
)

# ✅ Save the model manually (so it can be loaded later)
model.save("best_gru_model.keras") 

# ✅ 6. 評估（記得還原 expm1）
y_pred = model.predict(X_valid)
y_pred_restored = np.expm1(y_pred)
y_valid_restored = np.expm1(y_valid)

rmse = np.sqrt(mean_squared_error(y_valid_restored.flatten(), y_pred_restored.flatten()))
print(f"✅ Final Validation RMSE: {rmse:.4f}")


2025-05-04 03:23:27.693636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746329007.939407      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746329008.009350      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-04 03:23:43.023205: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead

Epoch 1/20
28134/28134 ━━━━━━━━━━━━━━━━━━━━ 420s 15ms/step - loss: 0.2625 - val_loss: 0.1228
Epoch 2/20
28134/28134 ━━━━━━━━━━━━━━━━━━━━ 433s 15ms/step - loss: 0.2384 - val_loss: 0.1209
Epoch 3/20
28134/28134 ━━━━━━━━━━━━━━━━━━━━ 429s 15ms/step - loss: 0.2350 - val_loss: 0.1224
Epoch 4/20
28134/28134 ━━━━━━━━━━━━━━━━━━━━ 428s 15ms/step - loss: 0.2331 - val_loss: 0.1216
Epoch 5/20
28134/28134 ━━━━━━━━━━━━━━━━━━━━ 412s 15ms/step - loss: 0.2320 - val_loss: 0.1213
Epoch 6/20
28134/28134 ━━━━━━━━━━━━━━━━━━━━ 415s 15ms/step - loss: 0.2311 - val_loss: 0.1219
Epoch 7/20
28134/28134 ━━━━━━━━━━━━━━━━━━━━ 423s 15ms/step - loss: 0.2302 - val_loss: 0.1233
4907/4907 ━━━━━━━━━━━━━━━━━━━━ 27s 5ms/step
✅ Final Validation RMSE: 0.7217


In [10]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# ✅ Load model
model = load_model("best_gru_model.keras")
expected_feature_count = model.input_shape[-1]
print(f"✅ Model expects input shape: (None, 28, {expected_feature_count})")

# ✅ Prepare features
train_feature_cols = [col for col in train_df.columns if col not in ['sales', 'date', 'd', 'id']]
non_numeric_cols = train_df[train_feature_cols].select_dtypes(include=['object']).columns.tolist()
if non_numeric_cols:
    print(f"🚨 Dropping non-numeric columns: {non_numeric_cols}")
    train_feature_cols = [col for col in train_feature_cols if col not in non_numeric_cols]

# ✅ Pad dummy features if needed
current_feature_count = len(train_feature_cols)
if current_feature_count < expected_feature_count:
    pad_count = expected_feature_count - current_feature_count
    for i in range(pad_count):
        pad_col = f"pad_{current_feature_count + i}"
        train_df[pad_col] = 0.0
        train_feature_cols.append(pad_col)
    print(f"⚠️ Padded with {pad_count} columns")

# ✅ Step 1: Predict F1–F28
X_input = train_df[train_feature_cols].values[-28:].astype("float32")
X_input = np.expand_dims(X_input, axis=0)  # shape: (1, 28, n_features)
y_pred_1 = np.expm1(model.predict(X_input, verbose=0)).flatten()

# ✅ Step 2: Use F1–F28 as next input for F29–F56
future_df = train_df.iloc[-28:].copy()
future_df['sales'] = y_pred_1
X_input_2 = future_df[train_feature_cols].values.astype("float32")
X_input_2 = np.expand_dims(X_input_2, axis=0)
y_pred_2 = np.expm1(model.predict(X_input_2, verbose=0)).flatten()

# ✅ Step 3: Create 1-row submission
y_full = np.concatenate([y_pred_1, y_pred_2])
submission_df = pd.DataFrame({
    "id": ["model_1"],
    **{f"F{i+1}": [val] for i, val in enumerate(y_full)}
})

# ✅ Match to actual Kaggle sample_submission.csv
sample = pd.read_csv("/kaggle/input/submission-v2/submission.csv")
final_submission = pd.concat([sample.drop(columns=sample.columns[1:]), submission_df.iloc[0:1].drop(columns=['id'])], axis=1)
final_submission["id"] = sample["id"]
final_submission = final_submission[sample.columns]  # keep order

# ✅ Save final
final_submission.to_csv("submission_final_household.csv", index=False)
print("🎯 Saved submission_final_household.csv with shape:", final_submission.shape)


✅ Model expects input shape: (None, 28, 26)
🎯 Saved submission_final_household.csv with shape: (30490, 29)


In [ ]:
# Load the correct features used during training
train_feature_cols = [col for col in train_df.columns if col not in ['sales', 'date', 'id', 'd']]
print("✅ Train features:", len(train_feature_cols))

# Now inspect your current input features
current_feature_cols = [col for col in df.columns if col not in ['sales', 'date', 'id', 'd'] and df[col].dtype in [np.float64, np.int64]]
print("📦 Current input features:", len(current_feature_cols))

# Compare them
missing = set(train_feature_cols) - set(current_feature_cols)
print("🚨 Missing:", missing)


In [11]:
import shutil

# 將 CSV 壓縮成 zip（Kaggle 支援下載 zip，比較穩）
shutil.make_archive("/kaggle/working/submission_household", 'zip', "/kaggle/working", "submission_final_household.csv")


'/kaggle/working/submission_household.zip'